# Tree Scout
In this notebook we will use the Global Forest Watch API of the past decade to predict future deforestation

In [21]:
# Import necessary packages
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import json
import warnings
warnings.filterwarnings('ignore')

# For environment variables
from dotenv import load_dotenv
import os

# For machine learning
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Load environment variables from .env file
load_dotenv()

print("All packages imported successfully!")

All packages imported successfully!


In [22]:
import csv 
import numpy as np



with open('US_County_Boundingboxes.csv', mode='r') as file:
    us_county_bounds_csv = csv.DictReader(file)

    county_boundries = {}
    i = 0
    for line in us_county_bounds_csv:
        xmin = float(line['xmin'])
        xmax = float(line['xmax'])
        ymin = float(line['ymin'])
        ymax = float(line['ymax'])
        county_boundries[line['COUNTY_NAME']] = [[[xmin, ymin], 
                                                 [xmin, ymax],
                                                 [xmax, ymin],
                                                 [xmax, ymax]]]

for key in county_boundries.keys():
    print(f"{key}: {county_boundries[key]}")



Autauga: [[[-86.921237, 32.307574], [-86.921237, 32.708213], [-86.411172, 32.307574], [-86.411172, 32.708213]]]
Baldwin: [[[-83.42909, 32.926141], [-83.42909, 33.1904], [-83.044327, 32.926141], [-83.044327, 33.1904]]]
Barbour: [[[-80.227173, 38.947236], [-80.227173, 39.303337], [-79.808791, 38.947236], [-79.808791, 39.303337]]]
Bibb: [[[-83.89205, 32.660643], [-83.89205, 32.952792], [-83.48943, 32.660643], [-83.48943, 32.952792]]]
Blount: [[[-84.188319, 35.460543], [-84.188319, 35.887125], [-83.662765, 35.460543], [-83.662765, 35.887125]]]
Bullock: [[[-85.999258, 31.880275], [-85.999258, 32.305289], [-85.410042, 31.880275], [-85.410042, 32.305289]]]
Butler: [[[-80.166472, 40.669233], [-80.166472, 41.173134], [-79.688705, 40.669233], [-79.688705, 41.173134]]]
Calhoun: [[[-81.279482, 38.611982], [-81.279482, 39.045182], [-80.98495, 38.611982], [-80.98495, 39.045182]]]
Chambers: [[[-95.018205, 29.49624], [-95.018205, 29.88994], [-94.353383, 29.49624], [-94.353383, 29.88994]]]
Cherokee: [[

In [28]:
import os
from dotenv import load_dotenv

load_dotenv()

def get_county_forest_loss(county_coords):
    response = requests.post(  # Changed from GET to POST
        "https://data-api.globalforestwatch.org/dataset/umd_tree_cover_loss/v1.9.1/query/json",
        headers={
            "x-api-key": os.getenv('GFW_API_KEY'),
            "Content-Type": "application/json"
        },
        json={  # Changed from data= to json=
            "sql": "SELECT SUM(area__ha) FROM results WHERE umd_tree_cover_loss__year = 2014",
            "geometry": {
                "type": "Polygon",
                "coordinates": county_coords
            }
        }
    )

    print(f"status code: {response.status_code}")
    if response.status_code == 200:
        print(response.json())
    else:
        print(f"Error: {response.text}")

for index, (key, value) in enumerate(county_boundries.items()):
    get_county_forest_loss(value)
    if index == 10:
        break

status code: 200
{'data': [{'area__ha': 756.9775400000001}], 'status': 'success'}
status code: 200
{'data': [{'area__ha': 383.52966}], 'status': 'success'}
status code: 200
{'data': [{'area__ha': 138.41534}], 'status': 'success'}
status code: 200
{'data': [{'area__ha': 353.06205}], 'status': 'success'}
status code: 200
{'data': [{'area__ha': 93.78819}], 'status': 'success'}
status code: 200
{'data': [{'area__ha': 1055.4309}], 'status': 'success'}
status code: 200
{'data': [{'area__ha': 40.87115}], 'status': 'success'}
status code: 200
{'data': [{'area__ha': 10.649429999999999}], 'status': 'success'}
status code: 200
{'data': [{'area__ha': 116.15257}], 'status': 'success'}
status code: 200
{'data': [{'area__ha': 2788.77696}], 'status': 'success'}
status code: 200
{'data': [{'area__ha': 655.34474}], 'status': 'success'}
